In [22]:
import sys
sys.path.insert(0, "..")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IA.utils import mapping, parameter_range
from scipy import stats
import math
import swifter
from collections import Counter

In [2]:
!ls -lah analysis/not_uploaded/IA/ | grep class

-rw-r--r-- 1 janpf janpf 712M Oct 27 01:05 .scratch.ckpts.IA.pexels.scores-one.change_class.epoch-5.pth.txt
-rw-r--r-- 1 janpf janpf 836M Oct 27 01:05 .scratch.ckpts.IA.pexels.scores-three.change_class.epoch-1.pth.txt


In [3]:
df = pd.read_csv("analysis/not_uploaded/IA/.scratch.ckpts.IA.pexels.scores-one.change_class.epoch-5.pth.txt")

for col in df.columns:
    if not"score" in col:
        continue

    df[col] = df[col].swifter.apply(lambda row: eval(row)[0])

print(df.columns)
df

Pandas Apply: 100%|██████████| 2610000/2610000 [00:22<00:00, 114668.04it/s]
Index(['img', 'distortion', 'level', 'score', 'styles_contrast_class',
       'styles_brightness_class', 'styles_saturation_class',
       'styles_shadows_class', 'styles_highlights_class',
       'styles_exposure_class', 'styles_vibrance_class',
       'styles_temperature_class', 'styles_tint_class',
       'technical_jpeg_compression_class', 'technical_defocus_blur_class',
       'technical_motion_blur_class', 'technical_pixelate_class',
       'technical_gaussian_noise_class', 'technical_impulse_noise_class',
       'composition_rotate_class', 'composition_hcrop_class',
       'composition_vcrop_class', 'composition_leftcornerscrop_class',
       'composition_rightcornerscrop_class', 'composition_ratio_class'],
      dtype='object')


,img,distortion,level,score,styles_contrast_class,styles_brightness_class,styles_saturation_class,styles_shadows_class,styles_highlights_class,styles_exposure_class,...,technical_motion_blur_class,technical_pixelate_class,technical_gaussian_noise_class,technical_impulse_noise_class,composition_rotate_class,composition_hcrop_class,composition_vcrop_class,composition_leftcornerscrop_class,composition_rightcornerscrop_class,composition_ratio_class
0,5/3/c/abbey-glass-religion-architecture-106158...,original,0.0,0.830078,3.623047,5.226562,7.175781,6.683594,7.171875,2.625000,...,0.398193,0.854980,1.997070,0.702148,2.726562,2.298828,2.359375,2.263672,2.322266,0.961914
1,9/4/4/abraham-lincoln-lincoln-memorial-washing...,original,0.0,0.570312,7.121094,9.171875,10.195312,8.843750,10.031250,9.453125,...,-0.735840,2.380859,0.815918,-0.728516,4.417969,3.250000,3.578125,3.154297,3.255859,3.960938
2,a/5/9/abstract-background-backgrounds-botany-8...,original,0.0,0.374756,7.789062,7.332031,7.851562,8.062500,7.734375,7.847656,...,0.638672,3.476562,-3.263672,-6.703125,4.800781,2.710938,2.888672,2.816406,2.835938,1.827148
3,2/2/c/adler-bird-bird-of-prey-raptor-53587.jpeg,original,0.0,0.553711,6.363281,7.136719,8.328125,8.984375,8.359375,7.527344,...,1.218750,-0.740234,0.238525,1.049805,4.042969,2.613281,2.833984,2.494141,2.542969,3.861328
4,a/0/7/adult-cute-face-female-41522.jpeg,original,0.0,0.407471,11.007812,13.500000,12.304688,13.398438,12.343750,10.914062,...,2.169922,4.164062,0.513184,-4.781250,4.218750,5.000000,5.097656,5.125000,5.171875,3.882812
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609995,0/d/b/wood-painting-window-cactus-6689.jpg,ratio,5.0,0.716797,7.535156,8.476562,9.562500,7.472656,9.914062,8.281250,...,0.209473,1.983398,1.073242,0.865234,3.978516,3.611328,4.113281,3.267578,3.746094,2.777344
2609996,0/1/0/wood-sky-forest-autumn.jpg,ratio,5.0,0.522949,4.722656,5.097656,6.281250,6.027344,6.367188,3.958984,...,1.300781,2.632812,-2.576172,0.654297,2.146484,2.251953,2.152344,2.083984,2.695312,1.942383
2609997,0/d/2/wood-stool-wooden-wine.jpg,ratio,5.0,0.797363,7.339844,8.773438,8.523438,8.898438,9.304688,7.332031,...,1.437500,4.968750,-3.333984,-5.523438,3.738281,3.185547,3.212891,2.630859,3.322266,3.720703
2609998,b/5/e/wood-tree-brown-bark.jpg,ratio,5.0,0.852051,7.550781,6.988281,8.007812,8.078125,8.710938,5.902344,...,1.941406,3.630859,-2.933594,-5.585938,3.601562,3.058594,3.078125,2.585938,2.724609,2.375000


In [4]:
def find_class(row:pd.Series, dist_type:str):
    relevant_cols = [c for c, _ in row.iteritems() if dist_type in c and "class" in c]
    m = -1
    m_c = ""
    for c in relevant_cols:
        if row[c] > m:
            m = row[c]
            m_c = c
    return m_c.split("_")[1]

In [5]:
df["styles_class"] = df.apply(lambda row: find_class(row, "styles"), axis=1)
df["technical_class"] = df.apply(lambda row: find_class(row, "technical"), axis=1)
df["composition_class"] = df.apply(lambda row: find_class(row, "composition"), axis=1)

In [29]:
df[df["technical_class"] == "jpeg"]["technical_class"] = "jpeg_compression"

In [31]:
df

,img,distortion,level,score,styles_contrast_class,styles_brightness_class,styles_saturation_class,styles_shadows_class,styles_highlights_class,styles_exposure_class,...,technical_impulse_noise_class,composition_rotate_class,composition_hcrop_class,composition_vcrop_class,composition_leftcornerscrop_class,composition_rightcornerscrop_class,composition_ratio_class,styles_class,technical_class,composition_class
0,5/3/c/abbey-glass-religion-architecture-106158...,original,0.0,0.830078,3.623047,5.226562,7.175781,6.683594,7.171875,2.625000,...,0.702148,2.726562,2.298828,2.359375,2.263672,2.322266,0.961914,saturation,gaussian,rotate
1,9/4/4/abraham-lincoln-lincoln-memorial-washing...,original,0.0,0.570312,7.121094,9.171875,10.195312,8.843750,10.031250,9.453125,...,-0.728516,4.417969,3.250000,3.578125,3.154297,3.255859,3.960938,saturation,jpeg,rotate
2,a/5/9/abstract-background-backgrounds-botany-8...,original,0.0,0.374756,7.789062,7.332031,7.851562,8.062500,7.734375,7.847656,...,-6.703125,4.800781,2.710938,2.888672,2.816406,2.835938,1.827148,temperature,pixelate,rotate
3,2/2/c/adler-bird-bird-of-prey-raptor-53587.jpeg,original,0.0,0.553711,6.363281,7.136719,8.328125,8.984375,8.359375,7.527344,...,1.049805,4.042969,2.613281,2.833984,2.494141,2.542969,3.861328,shadows,jpeg,rotate
4,a/0/7/adult-cute-face-female-41522.jpeg,original,0.0,0.407471,11.007812,13.500000,12.304688,13.398438,12.343750,10.914062,...,-4.781250,4.218750,5.000000,5.097656,5.125000,5.171875,3.882812,brightness,pixelate,rightcornerscrop
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2609995,0/d/b/wood-painting-window-cactus-6689.jpg,ratio,5.0,0.716797,7.535156,8.476562,9.562500,7.472656,9.914062,8.281250,...,0.865234,3.978516,3.611328,4.113281,3.267578,3.746094,2.777344,temperature,jpeg,vcrop
2609996,0/1/0/wood-sky-forest-autumn.jpg,ratio,5.0,0.522949,4.722656,5.097656,6.281250,6.027344,6.367188,3.958984,...,0.654297,2.146484,2.251953,2.152344,2.083984,2.695312,1.942383,tint,pixelate,rightcornerscrop
2609997,0/d/2/wood-stool-wooden-wine.jpg,ratio,5.0,0.797363,7.339844,8.773438,8.523438,8.898438,9.304688,7.332031,...,-5.523438,3.738281,3.185547,3.212891,2.630859,3.322266,3.720703,highlights,pixelate,rotate
2609998,b/5/e/wood-tree-brown-bark.jpg,ratio,5.0,0.852051,7.550781,6.988281,8.007812,8.078125,8.710938,5.902344,...,-5.585938,3.601562,3.058594,3.078125,2.585938,2.724609,2.375000,highlights,jpeg,rotate


In [28]:
dist_df = df[df["distortion"].isin(mapping["technical"].keys())][["distortion", "technical_class"]]
dist_df

,distortion,technical_class
410,jpeg_compression,motion
411,jpeg_compression,jpeg
412,jpeg_compression,pixelate
413,jpeg_compression,jpeg
414,jpeg_compression,pixelate
...,...,...
2609685,impulse_noise,impulse
2609686,impulse_noise,impulse
2609687,impulse_noise,impulse
2609688,impulse_noise,jpeg


In [36]:
sorted(dist_df["distortion"].unique())

['defocus_blur',
 'gaussian_noise',
 'impulse_noise',
 'jpeg_compression',
 'motion_blur',
 'pixelate']

In [37]:
sorted(dist_df["technical_class"].unique())

['defocus', 'gaussian', 'impulse', 'jpeg', 'motion', 'pixelate']

In [32]:
dist_df["found"] = dist_df.apply(lambda row: row["distortion"].split("_")[0] == row["technical_class"], axis=1)

In [33]:
Counter(dist_df["found"])

Counter({False: 280444, True: 169556})

In [38]:
450000/6

75000.0